In [1]:
# !pip install openai

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
api_key = os.getenv("oapi_key")
base_url = "https://api.openai.com/v1"
model = "gpt-3.5-turbo"

In [4]:
import tiktoken

In [5]:
tokens = tiktoken.encoding_for_model(model).encode("Hi My name is Waqas")
tokens

[13347, 3092, 836, 374, 29614, 80, 300]

In [6]:
len(tokens)

7

In [7]:
client = OpenAI(api_key=api_key)

In [8]:
m = [{"role":"system","content": "You are an helpful assitant"},
    {"role":"user","content": "Write any poem for me"}]

response = client.chat.completions.create(model = model, messages = m)
assistant = response.choices[0].message.content

In [9]:
print(assistant)

In the realm of words, let us explore,
A tapestry of thoughts, forevermore.
I'll craft a poem, just for you,
To lift your spirits, and help you through.

When days are tough, and burdens weigh,
Let these verses be your guiding ray.
For in each line, a gentle embrace,
To bring you solace, in every space.

In blooming fields of vibrant hue,
Beauty unfolds, as blessings accrue.
The sunlit sky, a canvas of dreams,
Where hope ignites, as the soul redeems.

As rivers dance and mountains rise,
Nature's symphony, a grand surprise.
Let these wonders, in verse, be found,
From soaring heights to sacred ground.

When love eludes, and hearts break,
Let these words, your heartache slake.
For in the rhymes, sweet remedy lies,
To heal the wounds and dry the eyes.

In the darkest night, when stars shine bright,
Let poetry be your guiding light.
Unlock the secrets of the universe's art,
Through this poem's rhythm, woven smart.

So take these lines, like whispered prayers,
Carry them with you, beyond lif

In [10]:
m.append({"role":"assistant", "content": assistant})

In [11]:
m

[{'role': 'system', 'content': 'You are an helpful assitant'},
 {'role': 'user', 'content': 'Write any poem for me'},
 {'role': 'assistant',
  'content': "In the realm of words, let us explore,\nA tapestry of thoughts, forevermore.\nI'll craft a poem, just for you,\nTo lift your spirits, and help you through.\n\nWhen days are tough, and burdens weigh,\nLet these verses be your guiding ray.\nFor in each line, a gentle embrace,\nTo bring you solace, in every space.\n\nIn blooming fields of vibrant hue,\nBeauty unfolds, as blessings accrue.\nThe sunlit sky, a canvas of dreams,\nWhere hope ignites, as the soul redeems.\n\nAs rivers dance and mountains rise,\nNature's symphony, a grand surprise.\nLet these wonders, in verse, be found,\nFrom soaring heights to sacred ground.\n\nWhen love eludes, and hearts break,\nLet these words, your heartache slake.\nFor in the rhymes, sweet remedy lies,\nTo heal the wounds and dry the eyes.\n\nIn the darkest night, when stars shine bright,\nLet poetry be

In [12]:
for d in m:
    print(d)

{'role': 'system', 'content': 'You are an helpful assitant'}
{'role': 'user', 'content': 'Write any poem for me'}
{'role': 'assistant', 'content': "In the realm of words, let us explore,\nA tapestry of thoughts, forevermore.\nI'll craft a poem, just for you,\nTo lift your spirits, and help you through.\n\nWhen days are tough, and burdens weigh,\nLet these verses be your guiding ray.\nFor in each line, a gentle embrace,\nTo bring you solace, in every space.\n\nIn blooming fields of vibrant hue,\nBeauty unfolds, as blessings accrue.\nThe sunlit sky, a canvas of dreams,\nWhere hope ignites, as the soul redeems.\n\nAs rivers dance and mountains rise,\nNature's symphony, a grand surprise.\nLet these wonders, in verse, be found,\nFrom soaring heights to sacred ground.\n\nWhen love eludes, and hearts break,\nLet these words, your heartache slake.\nFor in the rhymes, sweet remedy lies,\nTo heal the wounds and dry the eyes.\n\nIn the darkest night, when stars shine bright,\nLet poetry be your g

In [13]:
for d in m:
    print(d["content"])

You are an helpful assitant
Write any poem for me
In the realm of words, let us explore,
A tapestry of thoughts, forevermore.
I'll craft a poem, just for you,
To lift your spirits, and help you through.

When days are tough, and burdens weigh,
Let these verses be your guiding ray.
For in each line, a gentle embrace,
To bring you solace, in every space.

In blooming fields of vibrant hue,
Beauty unfolds, as blessings accrue.
The sunlit sky, a canvas of dreams,
Where hope ignites, as the soul redeems.

As rivers dance and mountains rise,
Nature's symphony, a grand surprise.
Let these wonders, in verse, be found,
From soaring heights to sacred ground.

When love eludes, and hearts break,
Let these words, your heartache slake.
For in the rhymes, sweet remedy lies,
To heal the wounds and dry the eyes.

In the darkest night, when stars shine bright,
Let poetry be your guiding light.
Unlock the secrets of the universe's art,
Through this poem's rhythm, woven smart.

So take these lines, like 

In [14]:
encoding = tiktoken.encoding_for_model(model)

In [15]:
token_counts = 0
for d in m:
    text = d["content"]
    token = encoding.encode(text) # token list will be returned
    total_token = len(token)
    token_counts += total_token 

In [16]:
token_counts

267

In [17]:
token_counts = sum(len(encoding.encode(d["content"])) for d in m)

In [18]:
token_counts

267

In [25]:
class ConversationBot:
    
    def __init__(self,api_key, 
                 base_url = base_url, 
                 model = model,
                system_message = """You are a helpful marketing assistant for DigiTech Synergy.""",
                 token_budget = 4096,
                ):
        self.client = OpenAI(api_key=api_key)
        self.client.base_url = base_url
        self.model = model
        self.system_message = system_message
        self.token_budget = token_budget
        self.conversation_history = [{"role":"system","content": self.system_message}]
        
    def count_token(self,text):
        try:
            encoding = tiktoken.encoding_for_model(self.model)
        except KeyError:
            encoding = tiktoken.encoding_for_model("cl100k_base")

        tokens = encoding.encode(text)
        return len(token)

    def total_tokens_used(self):
        token_counts = sum(self.count_token(d["content"]) for d in self.conversation_history)
        return token_counts

    def enforce_token_budget(self):
        while self.total_tokens_used() > self.token_budget:
            if len(self.conversation_history) <= 1: # only system message is available
                break

            self.conversation_history.pop(1)

            
    
    def chat_completion(self, prompt, temperature = 0.7, max_tokens = 500):
        
        self.enforce_token_budget()
        
        self.conversation_history.append({"role":"user", "content":prompt})
        
        response = self.client.chat.completions.create(model = self.model,
                                                       messages = self.conversation_history)
        ai_response = response.choices[0].message.content
        self.conversation_history.append({"role":"assistant", "content":ai_response})
        
        
        return ai_response

In [27]:
conv_manager = ConversationBot(api_key = api_key, token_budget=100)
prompt1 = "Please write a tweet to promote our new AI Industrial Training."
response = conv_manager.chat_completion(prompt1)

In [28]:
response

"Looking to level up your skills in the fast-paced world of AI? Look no further! Introducing our cutting-edge AI Industrial Training program that will equip you with the knowledge and hands-on experience you need. Don't miss out, sign up today! 💡🚀 #AItraining #industrialtech #learningopportunity"

In [29]:
conv_manager.count_token(response)

256

In [30]:
conv_manager.total_tokens_used()

768

In [31]:
prompt2 = "Great, now make the tweet a longer."
response2 = conv_manager.chat_completion(prompt2)

In [32]:
response2

'📣 Exciting news! @DigiTechSynergy is revolutionizing the world of digital technology with our cutting-edge solutions and unmatched expertise. 🌐🚀 Join us on our mission to redefine the future of tech and unlock endless possibilities. Stay tuned for game-changing innovations that will reshape industries and empower businesses like never before! 💪💡 #TechRevolution #InnovationUnleashed #FutureTechLeaders'

In [33]:
prompt3 = "Great, now add one more hash tag."

response3 = conv_manager.chat_completion(prompt3)